In [ ]:
# Installing libraries for traning of model
! pip install --upgrade deepforest albumentations pyyaml
#pip install git+https://github.com/weecology/DeepForest.git
!pip uninstall opencv-python-headless -y
!pip install opencv-python-headless==4.1.2.30
!pip install GDAL
!pip install matplotlib
!pip uninstall pytorch_lightning -y
!pip install pytorch_lightning

In [6]:
# Importing Libraries
from osgeo import gdal
import os
import numpy as np
from deepforest import main
from deepforest import get_data
from deepforest import utilities
from deepforest import preprocess
import matplotlib.pyplot as plt
from torch import optim, nn, utils, Tensor
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
import pytorch_lightning as pl

In [ ]:
# Storing data locally and managing it
!mkdir dataConversion
!cp -r OSBS_megaplot_tiles dataConversion

In [13]:
# Creating annotations from shapefile
def intoAnnotaions(shapefile, rgb, counter):
  df=utilities.shapefile_to_annotations(shapefile, rgb)
  df.to_csv("dataConversion/shapefile_annotation.csv")

  # Splitting rasters into small rasters with patch size of 100 and patch overlapping of 0.05
  df2=preprocess.split_raster(annotations_file="dataConversion/shapefile_annotation.csv", path_to_raster=rgb, numpy_image=None, base_dir='dataConversion', patch_size=400, patch_overlap=0.05, allow_empty=False, image_name=None)
  df2.to_csv(f"dataConversion/cropRaster_annotation{counter}.csv")

  for i in df2.xmin:
    if type(i) == type(2):
      pass
    else:
      print(i)

In [ ]:
# annotating the following data
intoAnnotaions(shapefile="OSBS_megaplots/2019_OSBS_5_404000_3285000_image.shp", rgb="OSBS_megaplot_tiles/2014_OSBS_1_404000_3285000_image.tif", counter=1)
intoAnnotaions(shapefile="OSBS_megaplots/2019_OSBS_5_404000_3284000_image.shp", rgb="OSBS_megaplot_tiles/2014_OSBS_1_404000_3284000_image.tif", counter=2)
intoAnnotaions(shapefile="OSBS_megaplots/2019_OSBS_5_403000_3285000_image.shp", rgb="OSBS_megaplot_tiles/2014_OSBS_1_403000_3285000_image.tif", counter=3)
intoAnnotaions(shapefile="OSBS_megaplots/2019_OSBS_5_403000_3284000_image.shp", rgb="OSBS_megaplot_tiles/2014_OSBS_1_403000_3284000_image.tif", counter=4)

In [ ]:
# Model configuration and training
from pytorch_lightning.loops import dataloader
import pandas as pd 
import torch

for i in range(4):

  annotationFile=f"/content/dataConversion/cropRaster_annotation{i+1}.csv"

  model = main.deepforest()

  model.config["gpus"] = "-1"
  model.config["train"]["epochs"] = 100
  model.config["epochs"] = 100
  model.config["train"]["csv_file"] = annotationFile
  model.config["score_thresh"] = 0.4
  model.config["train"]["root_dir"] = os.path.dirname(annotationFile)
  model.config["train"]["fast_dev_run"] = False

  # model.create_trainer()
  # model.use_release()
  # model.trainer.fit(model)

  trainer= pl.Trainer( max_epochs=100,
              gpus="-1",
              enable_checkpointing=False,
              accelerator='gpu',
              fast_dev_run= False)

  trainer.fit(model=model)